In [1]:
from imports import *

In [2]:
file_path = 'selected_features_PCA.xlsx'
df = pd.read_excel(file_path)
df.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,Irritation
0,-0.156202,0.917365,1.215465,0.378529,0.466241,0.209837,0.232062,0.536511,0.303403,0.196044,1
1,0.133787,0.361076,0.384876,0.713980,0.694891,0.375455,0.234795,0.536740,0.303088,0.201185,1
2,-2.006071,0.000000,1.841560,0.417235,0.459854,0.277571,0.130574,0.724242,0.000000,0.005127,1
3,-0.535903,0.607914,0.648805,0.619348,0.650299,0.287167,0.165618,0.464030,0.396366,0.071052,1
4,-1.577659,0.927894,1.240141,0.480844,0.000000,0.250357,0.112937,0.724242,0.000000,0.028049,1


In [44]:
# Define features (X) and target variable (y)
X = df.drop(columns=['Irritation'])  # Assuming 'Call' is the target variable
y = df['Irritation']

In [45]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Print the shapes of the resulting sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (353, 10)
Shape of X_test: (89, 10)
Shape of y_train: (353,)
Shape of y_test: (89,)


In [48]:
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def objective(trial):
    # Define hyperparameter search space
    param = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-4, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-4, 10.0, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 800),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 15),
        "gamma": trial.suggest_float("gamma", 1e-8, 5.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.9),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 5, 100),  # Moved here!
    }

    # Split train data into training and validation for early stopping
    X_train_, X_valid_, y_train_, y_valid_ = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

    # Train model with early stopping (now in constructor)
    model = xgb.XGBClassifier(**param, use_label_encoder=False)
    model.fit(
        X_train_, y_train_,
        eval_set=[(X_valid_, y_valid_)],  # Use validation data
        verbose=False
    )

    # Evaluate on validation set
    y_pred = model.predict(X_valid_)
    accuracy = accuracy_score(y_valid_, y_pred)

    return accuracy  # Optuna will maximize this

# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print best hyperparameters
print("Best trial:", study.best_trial.params)

# Train final model with best parameters
best_params = study.best_trial.params
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False)
final_model.fit(X_train, y_train)

# Evaluate on test set
y_pred_final = final_model.predict(X_test)
print("Final Model Accuracy:", accuracy_score(y_test, y_pred_final))


[I 2025-01-31 11:13:46,539] A new study created in memory with name: no-name-897a39b5-00ef-4077-ad92-8a8c397ecd8b
[I 2025-01-31 11:13:46,666] Trial 0 finished with value: 0.8113207547169812 and parameters: {'booster': 'gbtree', 'lambda': 0.00028651305251997715, 'alpha': 0.018072927516376902, 'learning_rate': 0.03704544185357371, 'n_estimators': 704, 'max_depth': 8, 'min_child_weight': 2, 'gamma': 0.27205219788638607, 'subsample': 0.5889694273600685, 'colsample_bytree': 0.6273792546453507, 'early_stopping_rounds': 89}. Best is trial 0 with value: 0.8113207547169812.
[I 2025-01-31 11:13:46,710] Trial 1 finished with value: 0.8301886792452831 and parameters: {'booster': 'gbtree', 'lambda': 0.01799006755619394, 'alpha': 0.0942470725405577, 'learning_rate': 0.08032033284845028, 'n_estimators': 123, 'max_depth': 8, 'min_child_weight': 8, 'gamma': 0.17201533595489002, 'subsample': 0.7580595069790005, 'colsample_bytree': 0.6344462663748307, 'early_stopping_rounds': 26}. Best is trial 1 with va

Best trial: {'booster': 'dart', 'lambda': 0.8180748125197252, 'alpha': 0.00032486338970744215, 'learning_rate': 0.0392286456084305, 'n_estimators': 596, 'max_depth': 6, 'min_child_weight': 7, 'gamma': 1.9688685252630635e-05, 'subsample': 0.8307903391602734, 'colsample_bytree': 0.8118477702802908, 'early_stopping_rounds': 13}


ValueError: Must have at least 1 validation dataset for early stopping.

In [11]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict, learning_curve, validation_curve

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, learning_curve


# Get best hyperparameters from study
best_params = study.best_trial.params
# Split train data into training and validation for early stopping
X_train_, X_valid_, y_train_, y_valid_ = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

# Initialize the model with best parameters and early stopping
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False)

final_model.fit(
        X_train_, y_train_,
        eval_set=[(X_valid_, y_valid_)],  # Use validation data
        verbose=False
)

# Evaluate final model
y_pred_final = final_model.predict(X_test)
print("Final Model Accuracy:", accuracy_score(y_test, y_pred_final))

# 1. Learning Curves with Early Stopping
train_sizes, train_scores, validation_scores = learning_curve(
    final_model, X_train, y_train, cv=5, n_jobs=-1, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
validation_mean = np.mean(validation_scores, axis=1)
validation_std = np.std(validation_scores, axis=1)

# Plot Learning Curve
plt.figure(figsize=(12, 6))
plt.plot(train_sizes, train_mean, label='Training accuracy', color='blue')
plt.plot(train_sizes, validation_mean, label='Validation accuracy', color='green')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
plt.fill_between(train_sizes, validation_mean - validation_std, validation_mean + validation_std, color='green', alpha=0.2)
plt.title('Learning Curve (Training vs Validation Accuracy)')
plt.xlabel('Training Size')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate accuracy on test data
test_accuracy = final_model.score(X_test, y_test)
train_accuracy = final_model.score(X_train, y_train)
print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


Final Model Accuracy: 0.8539325842696629


ValueError: 
All the 50 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/monika/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/monika/Library/Python/3.9/lib/python/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/Users/monika/Library/Python/3.9/lib/python/site-packages/xgboost/sklearn.py", line 1519, in fit
    self._Booster = train(
  File "/Users/monika/Library/Python/3.9/lib/python/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/Users/monika/Library/Python/3.9/lib/python/site-packages/xg

In [62]:
Best_trial: {'booster': 'gbtree', 'lambda': 0.0052103621382832265, 'alpha': 0.1620577811386946, 'learning_rate': 0.1170210607506638, 'n_estimators': 327, 'max_depth': 4, 'min_child_weight': 7, 'gamma': 1.6589810670976416e-05, 'subsample': 0.7843052779851918, 'colsample_bytree': 0.5079409289423329}


In [ ]:
Best_trial: {'booster': 'dart', 'lambda': 0.020998703887689537, 'alpha': 0.1390447886551301, 'learning_rate': 0.11229332078448975, 'n_estimators': 559, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 2.0219692829063354e-05, 'subsample': 0.7175177317110472, 'colsample_bytree': 0.8899318411369608}


In [ ]:
Best_trial: {'booster': 'dart', 'lambda': 3.9815594987137994, 'alpha': 1.1692306809315036, 'learning_rate': 0.08046698602150275, 'n_estimators': 700, 'max_depth': 6, 'min_child_weight': 10, 'gamma': 2.123245273357147e-05, 'subsample': 0.7970691270411416, 'colsample_bytree': 0.6813800155627162}

In [32]:
bp["n_estimators"]=500